In [1]:
import requests
url = 'https://raw.githubusercontent.com/Lambda-School-Labs/tagger-ds/master/notebooks/merged_data_for_KNN_with_body.csv'
myfile = requests.get(url)
open('email.csv', 'wb').write(myfile.content)

29340319

In [2]:
import pandas as pd

df = pd.read_csv("email.csv").drop(['negative.1', 'neutral.1', 'positive.1', 'compound.1',
       'final_pred.1'], axis=1)
print(df.shape) 
df.head(3)

(10000, 15)


,email_body,subject,content_type,to_who,from_who,dates,negative,neutral,positive,compound,final_pred,smart_tags,message_id,user_tags,thread_id
0,here is our forecast,Subject:,Content-Type: text/plain; charset=us-ascii,NaN,From: phillip.allen@enron.com,"Date: Mon, 14 May 2001 16:39:00 -0700 (PDT)",0.0,1.000,0.000,0.0000,3,['forecast'],1,NaN,NaN
1,traveling to have a business meeting takes the...,Subject: Re:,Content-Type: text/plain; charset=us-ascii,NaN,From: phillip.allen@enron.com,"Date: Fri, 4 May 2001 13:51:00 -0700 (PDT)",0.0,0.893,0.107,0.9313,5,"['travel', 'business', 'meeting', 'take', 'fun...",2,NaN,NaN
2,test successful way to go,Subject: Re: test,Content-Type: text/plain; charset=us-ascii,NaN,From: phillip.allen@enron.com,"Date: Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",0.0,0.513,0.487,0.5859,4,"['test', 'successful', 'way']",3,NaN,NaN


##Randon Forest Classifier

In [0]:
# Feature / target split
X = df['smart_tags']
y = df['final_pred']

In [0]:
# Do a train / test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [5]:
X_train.shape,y_train.shape, X_test.shape,y_test.shape

((7500,), (7500,), (2500,), (2500,))

In [0]:
from sklearn.ensemble import RandomForestClassifier
import spacy

nlp = spacy.load('en_core_web_sm')

# Vectorize
def embed(docs):
    return [nlp(doc).vector for doc in docs]

# model
classifier = RandomForestClassifier(n_estimators=100, random_state=107,
                                   max_depth=7)
# Fit the model on training data
classifier.fit(embed(X_train), y_train)

# Get predictions
pred = classifier.predict(embed(X_test))

In [8]:
classifier.score(embed(X_test), y_test)

0.7476

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()

In [0]:
from sklearn.decomposition import TruncatedSVD

# Singular value decomposition
svd = TruncatedSVD(n_components=100, # Just here for demo. 
                   algorithm='randomized',
                   n_iter=10)


# LSI: Latent semantic indexing
lsi = Pipeline([('vect', vect), ('svd', svd)])

In [0]:
from sklearn.model_selection import GridSearchCV
# Pipeline
pipe = Pipeline([
                 #Vectorrizer
                 ('vect', vect),
                 #Classifier
                 ('clf', rfc)
                ])

In [14]:
 
parameters = {
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:  5.3min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

In [0]:
accuracy = grid_search.best_score_

In [16]:
accuracy

0.8347999999999999

##KNN

In [17]:
#create Data Clean desc
tags = df['smart_tags']
tags

0                                            ['forecast']
1       ['travel', 'business', 'meeting', 'take', 'fun...
2                           ['test', 'successful', 'way']
3       ['randy', 'send', 'schedule', 'salary', 'level...
4                             ['let', 'shoot', 'tuesday']
                              ...                        
9995    ['s', 'pretty', 'sexy', 'huh', 'get', 'talk', ...
9996                            ['copy', 'idea', 'screw']
9997    [' ', 'forward', 'eric', 'basshouect', 'troy',...
9998                                ['buy', 'enron', 's']
9999    ['yea', 'thank', 'remind', 'bday', 'nearly', '...
Name: smart_tags, Length: 10000, dtype: object

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
#Count Vectorizer 
vector = CountVectorizer(stop_words='english')

# build vocab
vector.fit(tags)

# transform text
tags_vector = vector.transform(tags)

type(tags_vector.todense())

numpy.matrix

In [19]:
#matrix
tags_vector.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [20]:
#features name, Get Word Counts for each document
data_vector = pd.DataFrame(tags_vector.todense(), columns=vector.get_feature_names())
data_vector.head()

,aa,aaa,aachen,aactive,aadvantage,aaenhance,aagustincaisocom,aaiis,aallenenroncom,aamir,aand,aanoliladwpcom,aantie,aap,aapl,aaquila,aaron,aaronberuttienroncom,aaronjangenroncom,aaronson,aarp,aarti,aatkimageimag,ab,aba,abacus,abaginskicgauscgmil,abando,abandon,abandonment,abaout,abargeripccbvit,abate,abatement,abb,abbas,abbey,abbott,abbotts,abbreviate,...,zo,zoann,zoch,zochnaenronenron,zoes,zolfo,zolkoski,zonal,zoncom,zone,zoning,zool,zoolander,zoom,zoomerang,zora,zoran,zoya,zpro,zpros,zran,zranim,zschappel,zshop,zta,zub,zucha,zufferli,zufferlicalectect,zufferlienronenronxgate,zufferlihouectect,zuidholland,zukav,zuniga,zurich,zurichs,zuritahoueesees,zwiebel,zyftyahoocom,zzjaakkoahmalamorganstanleycom
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
#Attributes
result = []
for col in data_vector:
  x = sum(data_vector[col])
  result.append(x)

data = zip(vector.get_feature_names(), result)
words = pd.DataFrame(data, columns=['words', 'count'])

In [22]:
words['rank'] = words['count'].rank(method='first',ascending=True)
words.tail()

,words,count,rank
45697,zurichs,1,13539.0
45698,zuritahoueesees,4,28995.0
45699,zwiebel,2,18735.0
45700,zyftyahoocom,3,24228.0
45701,zzjaakkoahmalamorganstanleycom,1,13540.0


In [23]:
words[words['rank'] <=10]

,words,count,rank
2,aachen,1,1.0
4,aadvantage,1,2.0
5,aaenhance,1,3.0
12,aantie,1,4.0
13,aap,1,5.0
19,aaronson,1,6.0
21,aarti,1,7.0
22,aatkimageimag,1,8.0
25,abacus,1,9.0
27,abando,1,10.0


In [24]:
from sklearn.neighbors import NearestNeighbors

#Fit on DTM

nn = NearestNeighbors(n_neighbors=5, algorithm='kd_tree')
nn.fit(data_vector)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [25]:
nn.kneighbors([data_vector.iloc[0].values])

(array([[0., 0., 0., 0., 0.]]), array([[ 277, 2354, 1948,    0,  887]]))

In [26]:
#Query using kneighbors

nn.kneighbors([data_vector.iloc[3]])

(array([[0.        , 0.        , 0.        , 0.        , 3.60555128]]),
 array([[1957,    3, 1208, 2996, 1581]]))

In [27]:
#Query using kneighbors

nn.kneighbors([data_vector.iloc[10]])

(array([[0.        , 0.        , 0.        , 4.58257569, 4.79583152]]),
 array([[2971,   10, 1228, 9998, 9980]]))

In [35]:
tags[3][:200]

"['randy', 'send', 'schedule', 'salary', 'level', 'scheduling', 'group', 'plus', 'thought', 'change', 'need', 'patti', 's', 'example', 'phillip']"

In [36]:
sum(data_vector['salary'])

60